---
syncID: 
title: "Querying Location Data with NEON API and Python"
description: "Querying the 'locations/' NEON API endpoint with Python and navigating the response"
dateCreated: 2020-04-24
authors: Maxwell J. Burner
contributors: 
estimatedTime:
packagesLibraries: requests, json
topics: api
languagesTool: python
dataProduct: DP1.10003.001
code1: 
tutorialSeries: python-neon-api-series
urlTitle: neon_api_04_locations
---

In this tutorial we will learn about querying the 'locations/' enpoint of the NEON API using Python.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Query the locations endpoint of the NEON API for data on specific NEON locations
* Parse and navigate responses from the locations endpoint of the NEON API
* Get spatial and geolocation data about NEON sites and plots
* Navigate the parent-child relationships between NEON locations


### Install Python Packages

* **requests**
* **json**
* **pandas**



</div>

In this tutorial we will learn to use the *locations/* endpoint of the NEON API to get geoLocation information about NEON sites. While the previous tutorials all covered getting information from the *sites/*, *products/*, and *data/* endpoints, we will now begin exploring other endpoints of the NEON API.

Looking through some of the data we obtained in previous tutorials, you might have noticed that references to location tend to be vaguel; location names and labels are used without any real geospatial information such as the coordinates or size of the location. For more detail, we can use the *locations/* endpoint of the NEON API to get spatial information on specific locations used in NEON studies.

## Get a Named Location

The *locations/* endpoint is usually used to provide context to locations referenced in NEON data products. Let's start by looking at the bird count data for Lower Teakettle again. 

In [10]:
import requests
import json
import pandas as pd

In [11]:
#Define API call componenets
SERVER = 'http://data.neonscience.org/api/v0/'
SITECODE = 'TEAK'
PRODUCTCODE = 'DP1.10003.001'

Again, let's get the basic bird count data for June 2018.

In [12]:
#Request data file list for 2018-04
data_request = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2018-06')
data_json = data_request.json()

In [13]:
for file in data_json['data']['files']:
    if('count' in file['name']):
        if('basic' in file['name']):
            bird_url = file['url']

df_bird = pd.read_csv(bird_url)
print(df_bird.dtypes)

uid                    object
namedLocation          object
domainID               object
siteID                 object
plotID                 object
plotType               object
pointID                object
startDate              object
eventID                object
pointCountMinute        int64
targetTaxaPresent      object
taxonID                object
scientificName         object
taxonRank              object
vernacularName         object
observerDistance      float64
detectionMethod        object
visualConfirmation     object
sexOrAge               object
clusterSize           float64
clusterCode            object
identifiedBy           object
dtype: object



Notice the 'namedLocation' attribute/column in our dataframe. Let's look at some of the values.

In [14]:
df_bird['namedLocation'][0:5]

0    TEAK_010.birdGrid.brd
1    TEAK_010.birdGrid.brd
2    TEAK_010.birdGrid.brd
3    TEAK_010.birdGrid.brd
4    TEAK_010.birdGrid.brd
Name: namedLocation, dtype: object

## Querying Locations

We can query the location 'SJER_013.birdGrid.brd' using the *locations/* endpoint in the NEON API. This will allow us to obtain detailed geoSpatial data on that particular location.

In [15]:
loc_request = requests.get(SERVER+'locations/'+'TEAK_010.birdGrid.brd')
loc_json = loc_request.json()

In [16]:
#Let's look at the keys of the data dictionary
for key in loc_json['data'].keys():
    print(key)

locationDescription
locationName
locationType
domainCode
siteCode
locationDecimalLatitude
locationDecimalLongitude
locationElevation
locationUtmEasting
locationUtmNorthing
locationUtmHemisphere
locationUtmZone
alphaOrientation
betaOrientation
gammaOrientation
xOffset
yOffset
zOffset
locationProperties
locationParent
locationParentUrl
locationChildren
locationChildrenUrls


First off, the data includes some basic titles and categories for the location.

In [17]:
print('Description: ',loc_json['data']['locationDescription'])
print('Name: ',loc_json['data']['locationName'])
print('Type: ',loc_json['data']['locationType'])
print('Data: ',loc_json['data']['domainCode'])

Description:  Plot "TEAK_010" at site "TEAK"
Name:  TEAK_010.birdGrid.brd
Type:  OS Plot - brd
Data:  D17


The actual spatial location data includes latitude, longitude, elevation, and Universal Transverse Mercator (UTM) Coordinates.

In [18]:
print('Latitude: ',loc_json['data']['locationDecimalLatitude'])
print('Longitude: ',loc_json['data']['locationDecimalLongitude'])
print('Elevation: ',loc_json['data']['locationElevation'])
print('UTM Easting: ',loc_json['data']['locationUtmEasting'])
print('UTM Northing: ',loc_json['data']['locationUtmNorthing'])
print('Hemisphere: ', loc_json['data']['locationUtmHemisphere'])
print('UTM Zone: ', loc_json['data']['locationUtmZone'])

Latitude:  37.01395
Longitude:  -119.049086
Elevation:  2220.26
UTM Easting:  317702.905270038
UTM Northing:  4098382.78517289
Hemisphere:  N
UTM Zone:  11


Let's take a closer look at the 'location properties' section. This is a list of dictionaries, where each dict has one 'locationPropertyName' element and a corresponding 'locationPropertyValue' element. These together provide a more detailed description of the properties of the location.

In [19]:
#Print location property names and values
for locationProperty in loc_json['data']['locationProperties']:
    print(locationProperty['locationPropertyName'][9:], #trim 'Value for ' off beginning of each name
          ': ',locationProperty['locationPropertyValue'])

 Coordinate source :  Geo 7X (H-Star)
 Coordinate uncertainty :  0.45
 Country :  unitedStates
 County :  Fresno
 Elevation uncertainty :  0.53
 Filtered positions :  404.0
 Geodetic datum :  WGS84
 Horizontal dilution of precision :  0.9
 Maximum elevation :  2221.09
 Minimum elevation :  2150.52
 National Land Cover Database (2001) :  evergreenForest
 Plot dimensions :  500m x 500m
 Plot ID :  TEAK_010
 Plot size :  250000.0
 Plot subtype :  birdGrid
 Plot type :  distributed
 Positional dilution of precision :  1.6
 Reference Point Position :  B2
 Slope aspect :  285.26
 Slope gradient :  5.43
 Soil type order :  Entisols
 State province :  CA
 Subtype Specification :  ninePoints
 UTM Zone :  11N


Finally there's the name and URLs for the locations' parent location and children locations, if any. The parent location is a location of which the current location is a part, while the children locations are smaller areas of the current location; requesting data on the children locations through the *locations/* endpoint allows us to get spatial data on a finer resolution.

In [20]:
#Print name and API url of parent location
print(loc_json['data']['locationParent'], loc_json['data']['locationParentUrl'])

TEAK https://data.neonscience.org/api/v0/locations/TEAK


In [21]:
#Print names and API urls of child locations
for child in zip(loc_json['data']['locationChildren'], loc_json['data']['locationChildrenUrls']):
    print(child[0], child[1])

TEAK_010.birdGrid.brd.B2 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.B2
TEAK_010.birdGrid.brd.C3 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.C3
TEAK_010.birdGrid.brd.C2 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.C2
TEAK_010.birdGrid.brd.C1 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.C1
TEAK_010.birdGrid.brd.B3 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.B3
TEAK_010.birdGrid.brd.B1 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.B1
TEAK_010.birdGrid.brd.A1 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.A1
TEAK_010.birdGrid.brd.A2 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.A2
TEAK_010.birdGrid.brd.A3 https://data.neonscience.org/api/v0/locations/TEAK_010.birdGrid.brd.A3
